# Glosario Total Renting

In [1]:
# Cargamos librerias

import pandas as pd
import people_also_ask
from youtubesearchpython import VideosSearch
import re
import time
from numpy import random

In [2]:
# Improtamos en un DF los coches para la búsqueda

coches = pd.read_csv('https://raw.githubusercontent.com/mariopaj/Trabajo_TR/main/coches.csv') # Introducir ruta en donde esté ubicado el csv de las Key Words
  
coches = list(coches.Keyword.apply(lambda x: x.strip())) # Lo pasamos a una lista y eliminamos espacios

coches[:10]


['Abarth 500',
 'Alfa Romeo Giulia',
 'Alfa Romeo Stelvio',
 'Audi A1',
 'Audi A3',
 'Audi A4',
 'Audi A5',
 'Audi A6',
 'Audi A7',
 'Audi A8']

In [3]:
len(coches)

255

In [4]:
# Función que devuelve pregunta-respuesta en formato HTML

def obtener_preguntas_frecuentes_html(key_word, n_preguntas=5, search='Buscar'):
    
    '''
    Esta función sirve para sacar las preguntas frecuentes y su respectiva respuesta
    para cualquier key word
    
    Parameters
    ----------
    key_word: string
        key word de la que queremos encontrar las preguntas y respuestas frecuentes 
    
    n_preguntas: int
        número de preguntas máximas que queremos sacar de la key word, por defecto 5
    
    search: string
        palabra 'Buscar' en el idioma que vamos a buscar las palabras frecuentes, por defecto en español
    
    Returns
    -------
    string
        devuelve un string con las preguntas y las respuestas en formato HTML
    '''
    try:
        questions = people_also_ask.get_related_questions(key_word, n_preguntas)
        questions = [question[0:question.find(search)] for question in questions] # Sacamos las preguntas frecuentes limpias

        answers = []
        contenido = []
        for question in questions:
            try:
                if len(people_also_ask.get_answer(question)['response']) > 3 and people_also_ask.get_answer(question)['snippet_type'] == 'Definition Featured Snippet':    # Comprobamos si el tamaño de la respuesta es mayor a 3 y si la respuesta es una definición
                    answers.append(re.sub(r'[0-9]{1,2} [a-z]{3,4} [0-9]{4}', '', people_also_ask.get_answer(question)['response']))                                        # Eliminamos la fecha final de la respuesta                                                                                      # Cogemos la respuesta tal cual sale
                    contenido.append('<h3>' + question + '</h3>' + '<p>' + re.sub(r'[0-9]{1,2} [a-z]{3,4} [0-9]{4}', '', people_also_ask.get_answer(question)['response']) + '</p>')

                # Esta parte es por si la respuesta a la pregunta es una tabla
                elif len(people_also_ask.get_answer(question)['response']) > 3 and people_also_ask.get_answer(question)['snippet_type'] == 'Table Featured Snippet':       # Comprobamos si el tamaño de la respuesta es mayor a 3 y si la respuesta es una tabla
                    columns = ' '.join(people_also_ask.get_answer(question)['snippet_data']['columns'])                                                                    # Cogemos la respuesta de la columna
                    values = ' '.join([' '.join([str(item) for item in sublist]) + '<br>' for sublist in people_also_ask.get_answer(question)['snippet_data']['values']])  # Sacamos los values en una única lista
                    result = columns + '<br>' + values
                    answers.append(result)  
                    contenido.append('<h3>' + question + '</h3>' + '<p>' + result + '</p>')

                # Esta parte es por si la respuesta a la pregunta no es ni una tabla ni definition
                else:
                    answers.append('')
            except:
                answers.append('') # Si no encontramos respuesta añadimos un elemento vacío

        contenido = [''.join(contenido)][0]

        return contenido
    
    except:
        return print('There are no frequently asked questions')

In [5]:
# Función que devuelve pregunta-respuesta en formato Lista

def obtener_preguntas_frecuentes(key_word, n_preguntas=5, search='Buscar'):
    
    '''
    Esta función sirve para sacar las preguntas frecuentes y su respectiva respuesta
    para cualquier key word
    
    Parameters
    ----------
    key_word: string
        key word de la que queremos encontrar las preguntas y respuestas frecuentes 
    
    n_preguntas: int
        número de preguntas máximas que queremos sacar de la key word, por defecto 5
    
    search: string
        palabra 'Buscar' en el idioma que vamos a buscar las palabras frecuentes, por defecto en español
    
    Returns
    -------
    df
        devuelve un df de preguntas y respuestas
    '''
    try:
        questions = people_also_ask.get_related_questions(key_word, n_preguntas)
        questions = [question[0:question.find(search)] for question in questions] # Sacamos las preguntas frecuentes limpias

        answers = []
        for question in questions:
            try:
                if len(people_also_ask.get_answer(question)['response']) > 3 and people_also_ask.get_answer(question)['snippet_type'] == 'Definition Featured Snippet':    # Comprobamos si el tamaño de la respuesta es mayor a 3 y si la respuesta es una definición
                    answers.append(re.sub(r'[0-9]{1,2} [a-z]{3,4} [0-9]{4}', '', people_also_ask.get_answer(question)['response']))                                        # Eliminamos la fecha final de la respuesta                                                                                      # Cogemos la respuesta tal cual sale

                # Esta parte es por si la respuesta a la pregunta es una tabla
                elif len(people_also_ask.get_answer(question)['response']) > 3 and people_also_ask.get_answer(question)['snippet_type'] == 'Table Featured Snippet':       # Comprobamos si el tamaño de la respuesta es mayor a 3 y si la respuesta es una tabla
                    columns = ' '.join(people_also_ask.get_answer(question)['snippet_data']['columns'])                                                                    # Cogemos la respuesta de la columna
                    values = ' '.join([' '.join([str(item) for item in sublist]) for sublist in people_also_ask.get_answer(question)['snippet_data']['values']])  # Sacamos los values en una única lista
                    result = columns + values
                    answers.append(result)

                # Esta parte es por si la respuesta a la pregunta no es ni una tabla ni definition
                else:
                    answers.append('')
            except:
                answers.append('') # Si no encontramos respuesta añadimos un elemento vacío

            # Creamos una lista sólo de las preguntas que tienen respuesta de cada KeyWord
            question_answer = []
            for question, answer in zip(questions, answers):
                if answer != '':
                    question_answer.append([question, answer])
                else:
                    pass
            
            # Desglosamos la lista
            flat_list = []
            for q in question_answer:
                for item in q:
                    flat_list.append(item)
            
            flat_list.insert(0, key_word)
        
        return flat_list
    
    except:
        return print('There are no frequently asked questions')

In [6]:
# Sacamos el Excel de las preguntas y respuestas frecuentes

def preguntas_frecuentes_and_video_to_excel(kws, n=5, search='Buscar'):
    
    '''
    Esta función sirve para sacar las preguntas frecuentes y su respectiva respuesta
    para cualquier key word y almacenarlas en un excel
    
    Parameters
    ----------
    kws: list
        lista de keywords de las que queremos sacar las preguntas frecuentes
    
    n_preguntas: int
        número de preguntas máximas que queremos sacar de la key word, por defecto 5
    
    search: string
        palabra 'Buscar' en el idioma que vamos a buscar las palabras frecuentes, por defecto en español
    
    Returns
    -------
    df
        devuelve un df de preguntas y respuestas
    '''

    dffq = pd.DataFrame()
    for kw in kws:
        fq = obtener_preguntas_frecuentes(kw, n, search) # Sacamos la lista de pregunta-respuesta
        fq.insert(1, VideosSearch(kw, limit = 1).result()['result'][0]['link']) # insertamos el video en la segunda columna
        df = pd.DataFrame(fq).transpose()
        dffq = pd.concat([dffq, df])
        dffq.to_excel('glosario.xlsx', index = False) # Vamos almacenando los resultados en un excel (marcar la ruta donde quieres que se ubique el excel)
        
        print(kw)
        
        time.sleep(random.uniform(1, 2)) # Time sleep random entre 1 y 2
        
    return dffq.head()

In [7]:
preguntas_frecuentes_and_video_to_excel(coches[:4], 5, search = 'Search') # Prueba para varios coches

Abarth 500
Alfa Romeo Giulia
Alfa Romeo Stelvio
Audi A1


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,Abarth 500,https://www.youtube.com/watch?v=q2zRmgjry5w,How much does an Abarth 500 cost?,"The Abarths start at about $22,000. The Califo...",What is the difference between Abarth 595 and ...,"The Abarth 695 is a faster, more focussed vers...",Which Fiat Abarth is the fastest?,Abarth 695 Biposto Record,Is the Fiat 500 Abarth fast?,Engine 1.4-liter turbo Inline four-cylinderFro...,Is Fiat 500 Abarth fast?,The best version of the Abarth 500 has a top s...,NaN,NaN
0,Alfa Romeo Giulia,https://www.youtube.com/watch?v=qepFQDxES70,Who produced Alfa?,the Fiat Chrysler Automobile Group,Does Alfa Romeo make their own engines?,At the heart of every Alfa Romeo lies a precis...,Is Alfa Romeo made by Ferrari?,"Ferrari does not own Alfa Romeo, though the co...",Who makes the Giulia engine?,Alfa Romeo GiuliaPlatform Giorgio Related Alfa...,Are Alfa Romeo engines made by Ferrari?,The class-leading performance numbers of the A...,Who makes Alfa Romeo's?,the Fiat Chrysler Automobile Group
0,Alfa Romeo Stelvio,https://www.youtube.com/watch?v=2qqWoEeixJ4,Is the Alfa Romeo Stelvio a reliable car?,Alfa Romeo Stelvio reliability Alfa Romeo perf...,Is Alfa Romeo a good car?,The WhatCar? Survey awarded Alfa Romeo with a ...,Does Alfa Romeo have a lot of problems?,There are a few reasons why Alfa Romeos are co...,How much is a brand new Alfa Romeo Stelvio?,"$45,150",What problems do Alfa Romeo Stelvio have?,The automaker claims that the newest models ar...,How much is an Alfa Romeo Stelvio SUV?,How Much Does the Alfa Romeo Stelvio Cost? The...
0,Audi A1,https://www.youtube.com/watch?v=J4DvaEqCLw0,Is the Audi A1 coming to Canada?,Audi's A1 is a punchy hatchback we can't buy i...,Is Audi A1 expensive?,You'll pay a hefty price premium to own an Aud...,Is the Audi A1 going to be discontinued?,The Audi A1 will not have a direct successor w...,Is Audi A1 a luxury car?,The Audi A1 is a luxurious and sporty-looking ...,Is an Audi A1 good?,"It's also a good deal more fun to drive, but t...",NaN,NaN


In [10]:
# Traducimos el excel glosario a español

def traducir_glosario(route : str, language = 'ES'):
    
    '''
    Esta función sirve para traducir el glosario en formato excel al idioma que queramos
    
    Parameters
    ----------
    route: str
        Ruta en la que tenemos ubicado el excel glosario
    
    language: str
        Idioma al que queremos traducir el glosario
        
    
    Returns
    -------
    df
        devuelve el df traducido
    '''
    
    import deepl

    translator = deepl.Translator(str(input('Introduce la clave de autentificación de la API:'))) # Introducimos la clave de autentificacion para poder acceder a las traducciones

    df_original = pd.read_excel(route) # Importamos el excel escrapeado

    df_traduction = pd.DataFrame() # Creamos un DataFrame vacío
    df_traduction[0] = df_original[0] # Añadimos al DataFrame la KeyWord
    df_traduction[1] = df_original[1] # Añadimos al DataFrame el link del video

    for column in range(2, df_original.shape[1]):
        print(column)
        traduction = []
        for sentence in df_original[column]:
            try:
                traduction.append(translator.translate_text(sentence, target_lang=language).text)
            except:
                traduction.append('')

        df_traduction[column] = traduction
        df_traduction.to_excel('glosario_traducido.xlsx', index = False)

    return df_traduction

In [11]:
traducir_glosario('glosario.xlsx')

Introduce la clave de autentificación de la API: d2faa701-e799-87b0-5e80-a79f478a9df2:fx


2
3
4
5
6
7
8
9
10
11
12
13


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,Abarth 500,https://www.youtube.com/watch?v=q2zRmgjry5w,¿Cuánto cuesta un Abarth 500?,Los Abarth empiezan en unos 22.000 dólares. El...,¿Cuál es la diferencia entre el Abarth 595 y e...,El Abarth 695 es una versión más rápida y cent...,¿Qué Fiat Abarth es el más rápido?,Abarth 695 Biposto Record,¿Es rápido el Fiat 500 Abarth?,"Motor 1,4 litros turbo de cuatro cilindrosTama...",¿Es rápido el Fiat 500 Abarth?,La mejor versión del Abarth 500 tiene una velo...,,
1,Alfa Romeo Giulia,https://www.youtube.com/watch?v=qepFQDxES70,¿Quién produjo Alfa?,el Grupo Automovilístico Fiat Chrysler,¿Fabrica Alfa Romeo sus propios motores?,En el corazón de cada Alfa Romeo se encuentra ...,¿Es Alfa Romeo fabricado por Ferrari?,"Ferrari no es propietaria de Alfa Romeo, aunqu...",¿Quién fabrica el motor del Giulia?,Alfa Romeo GiuliaPlataforma Giorgio Relacionad...,¿Los motores de Alfa Romeo son fabricados por ...,Las cifras de rendimiento del Alfa Romeo Giuli...,¿Quién fabrica los Alfa Romeo?,el Grupo Automovilístico Fiat Chrysler
2,Alfa Romeo Stelvio,https://www.youtube.com/watch?v=2qqWoEeixJ4,¿Es el Alfa Romeo Stelvio un coche fiable?,Fiabilidad del Alfa Romeo Stelvio Alfa Romeo o...,¿Es Alfa Romeo un buen coche?,La encuesta WhatCar? otorgó a Alfa Romeo la qu...,¿Tiene Alfa Romeo muchos problemas?,Hay algunas razones por las que los Alfa Romeo...,¿Cuánto cuesta un Alfa Romeo Stelvio nuevo?,"$45,150",¿Qué problemas tiene el Alfa Romeo Stelvio?,El fabricante de automóviles afirma que los mo...,¿Cuánto cuesta un SUV Alfa Romeo Stelvio?,¿Cuánto cuesta el Alfa Romeo Stelvio? El Stelv...
3,Audi A1,https://www.youtube.com/watch?v=J4DvaEqCLw0,¿Va a llegar el Audi A1 a Canadá?,El A1 de Audi es un hatchback con garra que no...,¿Es caro el Audi A1?,Pagarás un precio elevado por tener un Audi A1...,¿Se va a dejar de fabricar el Audi A1?,El Audi A1 no tendrá un sucesor directo cuando...,¿Es el Audi A1 un coche de lujo?,El Audi A1 es un pequeño utilitario de aspecto...,¿Es bueno un Audi A1?,"También es mucho más divertido de conducir, pe...",,
